In [ ]:
import numpy as np
import os
import cv2
import h5py
import re
import sys
from glob import glob

sys.path.append(os.path.abspath(os.path.join("..")))
import parameters
from dataset_loader import create_pytorch_dataset

sys.path.remove(os.path.abspath(os.path.join("..")))

window_len = parameters.window_len
stride = parameters.stride
fair_comparison = parameters.fair_comparison
TOD = parameters.TOD

device = parameters.device
key_frame_extraction = parameters.key_frame_extraction
key_frame_extraction_algorithm = parameters.key_frame_extraction_algorithm
feature_extraction = parameters.feature_extraction
background_subtraction = parameters.background_subtraction
background_subtraction_algorithm = parameters.background_subtraction_algorithm
background_subtraction_algorithms = parameters.background_subtraction_algorithms
data_augmentation = parameters.data_augmentation
anomaly_detection_model = parameters.anomaly_detection_model

frame_rate_adjusted_dataset = parameters.frame_rate_adjusted_dataset
dataset_category = parameters.dataset_category
project_directory = parameters.project_directory
dataset_directory = parameters.dataset_directory
ht = parameters.ht
wd = parameters.wd

display_ht = 450
display_wd = 450
# Video can be 4fps / 8fps / 20fps. So each frame can be displayed for 250 ms / 125 ms / 50 ms
ms_per_frame = 125  # millisecond per frame

In [ ]:
# To display Preprocessed Video and Input Video
def display_videos(name, dset, path, vid_folder, input_video):

    # Extract if fall folder or ADL folder
    dir_type = re.findall("[a-zA-Z]+", vid_folder)[0]

    # Original Video
    vid_location = "{}\Dataset\Fall-Data\{}\{}\{}\{}".format(
        dataset_directory, dataset_category, dset, dir_type, vid_folder
    )
    vid_location = glob(vid_location + "/*.jpg") + glob(vid_location + "/*.png")
    vid_location.sort(key=lambda var: [int(x) if x.isdigit() else x for x in re.findall(r"[^0-9]|[0-9]+", var)])
    original_video = []
    for filename in vid_location:
        img = cv2.imread(filename, cv2.IMREAD_ANYCOLOR)
        if img is not None:
            original_video.append(img)

    # Preprocessed Video
    with h5py.File(path, "r") as hf:
        data_dict = hf["{}/Processed/Split_by_video".format(name)]
        preprocessed_video = data_dict[vid_folder]["Data"][:]

    # Input Video
    input_video = input_video

    # Length will not be equal. (Key Frame Extraction must be true)
    # print(len(original_video), len(preprocessed_video), len(input_video))

    print("Original Length -", len(original_video))
    print("Length after Key Frame Extraction -", len(input_video))

    # Display frames of the original and preprocessed video
    for index in range(len(original_video)):
        original_frame = cv2.resize(original_video[index], (display_ht, display_wd))
        # uint8 to float32, scale down by 255
        original_frame = (np.array(original_frame, dtype=np.float32)) / 255

        preprocessed_frame = cv2.resize(preprocessed_video[index], (display_ht, display_wd))
        # Only has height, width. So add a dimension for channel
        preprocessed_frame = np.expand_dims(preprocessed_frame, axis=-1)
        # float64 to float32
        preprocessed_frame = np.array(preprocessed_frame, dtype=np.float32)
        # Convert image from greyscale to RGB (To obtain 3 channels)
        preprocessed_frame = cv2.cvtColor(preprocessed_frame, cv2.COLOR_GRAY2RGB)

        horizontal_concatenation = np.concatenate([original_frame, preprocessed_frame], axis=1)

        cv2.imshow("Original, Preprocessed", horizontal_concatenation)
        k = cv2.waitKey(ms_per_frame) & 0xFF
        # Exit on 'esc' key
        if k == 27:
            break

    cv2.destroyAllWindows()
    cv2.waitKey(3000)  # Wait 3 seconds

    # Display frames of the input video (Key Frame Extracted)
    for index in range(len(input_video)):
        input_frame = cv2.resize(input_video[index], (display_ht, display_wd))
        # Only has height, width. So add a dimension for channel
        input_frame = np.expand_dims(input_frame, axis=-1)
        # uint8 to float32
        input_frame = np.array(input_frame, dtype=np.float32)
        # Convert image from greyscale to RGB (To obtain 3 channels)
        input_frame = cv2.cvtColor(input_frame, cv2.COLOR_GRAY2RGB)

        cv2.imshow("Key Frame - Input", input_frame)
        k = cv2.waitKey(ms_per_frame) & 0xFF
        # Exit on 'esc' key
        if k == 27:
            break

    cv2.destroyAllWindows()


def demo_pipeline_key_frame_extraction(name, dset, path):
    (
        Test_Dataset,
        test_dataloader,
        Train_Dataset,
        train_dataloader,
        x_data_test,
        y_data_test,
        x_info_test,
        x_data_train,
        y_data_train,
        x_info_train,
    ) = create_pytorch_dataset(name, dset, path, window_len, fair_comparison, stride, TOD)

    print("Key Frame Extraction - {}".format(key_frame_extraction))
    if key_frame_extraction:
        print("Key Frame Extraction Algorithm - {}".format(key_frame_extraction_algorithm))
    print("Feature Extraction - {}".format(feature_extraction))
    if feature_extraction:
        print("Background Subtraction - {}".format(background_subtraction))
        if background_subtraction:
            print("Background Subtraction Algorithm - {}".format(background_subtraction_algorithm))
    print("Frame rate adjusted dataset - {}".format(frame_rate_adjusted_dataset))
    print("{} Demo Train Videos - {}".format(dset, len(train_dataloader)))
    print("{} Demo Test Videos - {}".format(dset, len(test_dataloader)))

    for loader_index, dataloader in enumerate([test_dataloader, train_dataloader]):
        for i, (sample, labels) in enumerate(dataloader):

            # sample and labels are in windowed format

            # Test Dataloader
            if loader_index == 0:
                vid_folder = x_info_test[i]
                input_video = x_data_test[i]  # input_video is not windowed.
            # Train Dataloader
            else:
                vid_folder = x_info_train[i]
                input_video = x_data_train[i]  # input_video is not windowed.

            print("")
            print((vid_folder))

            # Input Video - Input
            display_videos(name, dset, path, vid_folder, input_video)


# Unimodality
list_of_files = ["Thermal", "ONI_IR", "IP"]
list_of_datasets = ["Thermal_T3", "ONI_IR_T", "IP_T"]

modality_index = 0  # 0 to 2

dset = list_of_files[modality_index]
name = list_of_datasets[modality_index]
path = "{}\Dataset\H5PY\{}_Data_set-{}-imgdim64x64.h5".format(project_directory, dataset_category, name)

demo_pipeline_key_frame_extraction(name, dset, path)